In [ ]:
!pip install pycytominer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 43.6 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 828.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 153.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycytominer import feature_select
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, accuracy_score, RocCurveDisplay
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Flatten,
    Activation, BatchNormalization, ReLU,
    Conv1D, Conv2D, AveragePooling1D, MaxPooling1D,GlobalAveragePooling1D,Add, TimeDistributed
)
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall
from sklearn.ensemble import RandomForestClassifier


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print("train :", train.shape)
print("test :", test.shape)
print("train colonne", train.columns)
print("test colonne", test.columns)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
train.describe()

#Visualisation

In [ ]:
sns.countplot(x='Activity',data=train)

In [ ]:
colonnes = list(train.columns)
colonnes = colonnes[1:]
print(colonnes)

#Sélection des features

In [ ]:
train[colonnes].corr()

In [ ]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "drop_na_columns",
]
na_cut = 0.05
corr_threshold = 0.95
output_dir = "profiles"

profile_df = feature_select(
                profiles=train,#donner nom df entier
                operation=feature_select_ops,
                features=colonnes,  # liste des features
                na_cutoff=na_cut,
                corr_threshold=corr_threshold
            )
selected_features = list(set(colonnes) & set(profile_df.columns))
print(selected_features)
print(len(selected_features))

In [ ]:
sns.heatmap(train[selected_features].corr(), cmap="coolwarm")
plt.show()

#Normalisation des données

In [ ]:
test_normalize = normalize(test[selected_features], norm='l2')
test_normalize = pd.DataFrame(test_normalize, columns=selected_features,index=test.index)
train_normalize=normalize(train[selected_features], norm='l2')
train_normalize = pd.DataFrame(train_normalize, columns=selected_features, index=train.index)
train_normalize["Activity"] = train["Activity"]

In [ ]:
test_normalize.head()


In [ ]:
train_normalize.head()

#Visualisation T-sne

In [ ]:
m = TSNE(learning_rate="auto")
tsne_feature = m.fit_transform(train_normalize)
train_normalize["x"] = tsne_feature[:, 0]
train_normalize["y"] = tsne_feature[:, 1]


In [ ]:
sns.scatterplot(x="x", y="y", hue="Activity",data=train_normalize)
plt.show()

Le graphique montre que les features sélectionnées permettent de bien discriminer les molécules en fonction de l'activité, on a deux groupes bien distinct

# Séparation train-test

In [ ]:
Y = train_normalize["Activity"]
X = train_normalize.drop(["Activity", "x", "y"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
def specificity(y_true, y_pred):
    y_pred = tf.round(y_pred)
    y_true = tf.cast(y_true, tf.float32)
    tn = tf.reduce_sum((1 - y_true) * (1 - y_pred))
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    return tn / (tn + fp + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred):
    y_pred = tf.round(y_pred)
    y_true = tf.cast(y_true, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())

    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))



#Modèle dense



In [ ]:
def create_model_dense(input_dim):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_dim,),
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(32, activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            Precision(name="precision"),
            Recall(name="recall"),
            specificity,
            f1_score
        ]
    )
    return model


In [ ]:
model_dense = create_model_dense(X_train.shape[1])
model_dense.summary()

In [ ]:
from keras.utils import plot_model

plot_model(model_dense, to_file="model_dense.png", show_shapes=True, show_layer_names=True)

In [ ]:
history_dense = model_dense.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test,y_test))

In [ ]:
epochs = 100
batch_size = 100
n_splits = 5
cv_accuracy = []
cv_precision = []
cv_recall = []
cv_specificity = []
cv_f1_score = []

X_reset = X.reset_index(drop=True)
y_reset = Y.reset_index(drop=True)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_reset)):
    print(f"\n===== Fold {fold+1} =====")

    # Séparation des données
    X_tr, X_val = X_reset.iloc[train_idx], X_reset.iloc[val_idx]
    y_tr, y_val = y_reset.iloc[train_idx], y_reset.iloc[val_idx]
    model = create_model_dense(X.shape[1])

    # Entraînement
    history_CV = model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Évaluation sur le fold
    scores_CV = model.evaluate(X_val, y_val, verbose=0)
    loss, acc, prec, rec, spec, f1 = scores_CV
    print(f"Fold {fold+1} - Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, Spec: {spec:.4f}, F1: {f1:.4f}") s
    cv_accuracy.append(acc)
    cv_precision.append(prec)
    cv_recall.append(rec)
    cv_specificity.append(spec)
    cv_f1_score.append(f1)


# Conserver l’historique du dernier fold
history = history_CV
# Résultats finaux
print("\n===== Résultats finaux =====")
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cv_accuracy)*100, np.std(cv_accuracy)*100))
print("Precision: %.2f%% (+/- %.2f%%)" % (np.mean(cv_precision)*100, np.std(cv_precision)*100))
print("Recall: %.2f%% (+/- %.2f%%)" % (np.mean(cv_recall)*100, np.std(cv_recall)*100))
print("Specificity: %.2f%% (+/- %.2f%%)" % (np.mean(cv_specificity)*100, np.std(cv_specificity)*100))
print("F1 Score: %.2f%% (+/- %.2f%%)" % (np.mean(cv_f1_score)*100, np.std(cv_f1_score)*100))

In [ ]:
#  "Accuracy"
plt.plot(history_dense.history['accuracy'])
plt.plot(history_dense.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history_dense.history['loss'])
plt.plot(history_dense.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#Modèle convolution 1D

In [ ]:
def create_conv1d(input_dim):
    model = Sequential()
    # Bloc 1
    model.add(Conv1D(filters=64, kernel_size=7, padding='same', activation='relu',
                     input_shape=(input_dim, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    # Bloc 2
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    # Bloc 3
    model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # Global pooling
    model.add(GlobalAveragePooling1D())
    # Dense layers
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    # Sortie binaire
    model.add(Dense(1, activation='sigmoid'))
    # Compilation
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            Precision(name="precision"),
            Recall(name="recall"),
            specificity,
            f1_score
        ]
    )

    return model

In [ ]:
model_conv1D = create_conv1d(X_train.shape[1])
model_conv1D.summary()

In [ ]:
history_conv1D = model_conv1D.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test,y_test))

In [ ]:
epochs = 100
batch_size = 100
n_splits = 5
# Listes pour stocker les métriques par fold
cv_accuracy = []
cv_precision = []
cv_recall = []
cv_specificity = []
cv_f1_score = []

X_reset = X.reset_index(drop=True)
y_reset = Y.reset_index(drop=True)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_reset)):
    print(f"\n===== Fold {fold+1} =====")

    # Séparation des données
    X_tr, X_val = X_reset.iloc[train_idx], X_reset.iloc[val_idx]
    y_tr, y_val = y_reset.iloc[train_idx], y_reset.iloc[val_idx]
    model = create_conv1d(X.shape[1])

    # Entraînement
    history_CV = model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Évaluation sur le fold
    scores_CV = model.evaluate(X_val, y_val, verbose=0)
    loss, acc, prec, rec, spec, f1 = scores_CV
    print(f"Fold {fold+1} - Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, Spec: {spec:.4f}, F1: {f1:.4f}") # Print all metrics
    cv_accuracy.append(acc)
    cv_precision.append(prec)
    cv_recall.append(rec)
    cv_specificity.append(spec)
    cv_f1_score.append(f1)


# Conserver l’historique du dernier fold
history = history_CV
# Résultats finaux
print("\n===== Résultats finaux =====")
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cv_accuracy)*100, np.std(cv_accuracy)*100))
print("Precision: %.2f%% (+/- %.2f%%)" % (np.mean(cv_precision)*100, np.std(cv_precision)*100))
print("Recall: %.2f%% (+/- %.2f%%)" % (np.mean(cv_recall)*100, np.std(cv_recall)*100))
print("Specificity: %.2f%% (+/- %.2f%%)" % (np.mean(cv_specificity)*100, np.std(cv_specificity)*100)) # Print specificity
print("F1 Score: %.2f%% (+/- %.2f%%)" % (np.mean(cv_f1_score)*100, np.std(cv_f1_score)*100)) # Print f1_score

In [ ]:
#  "Accuracy"
plt.plot(history_conv1D.history['accuracy'])
plt.plot(history_conv1D.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history_conv1D.history['loss'])
plt.plot(history_conv1D.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

#Evaluations modèles regloss conv1D



In [ ]:
benchmark = pd.read_csv("svm_benchmark.csv")
benchmark.head()
print(benchmark.shape)
print("test :", test.shape)


In [ ]:
predict_regloss = model_dense.predict(test_normalize)
pred_labels = (predict_regloss > 0.5).astype(int)
benchmark['TrueLabel'] = (benchmark['PredictedProbability'] > 0.5).astype(int)
y_pred = pred_labels
y_true = benchmark['TrueLabel']
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap='Reds')
plt.show()
#metrique
TN, FP, FN, TP = cm.ravel()
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
# precision = recall
precision = TP / (TP + FP)
F1_score = (2*TP)/ ((2*TP) + FP + FN)
print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score {F1_score:.4f}")
#Courbe ROC
y_score = predict_regloss.ravel()
roc_display = RocCurveDisplay.from_predictions(y_true, y_score)
plt.plot([0,1],[0,1],'k--')
plt.show()

#AUC
auc = roc_auc_score(y_true, y_score)
print(f"AUC: {auc:.4f}")


In [ ]:
predict_conv1D = model_conv1D.predict(test_normalize)
pred_labels_conv1D = (predict_conv1D > 0.5).astype(int)
benchmark['TrueLabel'] = (benchmark['PredictedProbability'] > 0.5).astype(int)
y_pred_conv1D = pred_labels_conv1D
y_true = benchmark['TrueLabel']
cm_conv1D = confusion_matrix(y_true, y_pred_conv1D)
disp_conv1D = ConfusionMatrixDisplay(confusion_matrix=cm_conv1D, display_labels=[0, 1])
disp_conv1D.plot(cmap='Reds')
plt.show()

TN, FP, FN, TP = cm_conv1D.ravel()
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
# precision = recall
precision = TP / (TP + FP)
F1_score = (2*TP)/ ((2*TP) + FP + FN)
print(f"Accuracy: {accuracy:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score {F1_score:.4f}")


#RF 1st learning

In [ ]:
def create_model_regloss(input_dim):
    inputs = Input(shape=(input_dim,)) # remove the extra dimension here
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs) # assign the model here
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
#modèle dense
model = create_model_regloss(X_train.shape[1])
# Apprentissage classique
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
encoder = Model(inputs=model.input, outputs=model.get_layer(index=-4).output)

In [ ]:
#modèle dense
# On récupère les embeddings appris par l’encodeur
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)
print("Shape des embeddings :", X_train_encoded.shape)

In [ ]:
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(X_train_encoded, y_train)
y_pred = rf.predict(X_test_encoded)

# Encode X_val before predicting
X_val_encoded = encoder.predict(X_val)
y_val_pred = rf.predict(X_val_encoded)

print("Accuracy (val):", accuracy_score(y_val, y_val_pred))

# Evaluation RF

In [ ]:
# 1. Encoder les données de test avec ton CNN
X_test_encoded = encoder.predict(test_normalize)

# 2. Prédictions avec le RandomForest
y_pred_rf = rf.predict(X_test_encoded)
y_true = benchmark['TrueLabel'].values

cm_rf= confusion_matrix(y_true, y_pred_rf)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=[0, 1])
disp_rf.plot(cmap='Reds')
plt.show()

# 4. Calcul des métriques
TN, FP, FN, TP = cm_rf.ravel()
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
precision   = TP / (TP + FP) if (TP + FP) > 0 else 0
F1_score    = (2*TP) / ((2*TP) + FP + FN) if ((2*TP) + FP + FN) > 0 else 0

print(f"Accuracy:   {accuracy:.4f}")
print(f"Sensitivity:{sensitivity:.4f}")
print(f"Specificity:{specificity:.4f}")
print(f"Precision:  {precision:.4f}")
print(f"F1-score:   {F1_score:.4f}")


#DNN, 2nd learning

In [ ]:
epoch = 20
batch_size = 50
learning = 0.001
dropout = 0.2

In [ ]:
def dnn_model(learning, dropout, input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(dropout),

        layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4)),
        layers.BatchNormalization(),
        layers.Dropout(dropout),

        layers.Dense(1, activation='sigmoid')
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
model_dnn = dnn_model(learning, dropout,X_train.shape[1])
model_dnn.summary()

In [ ]:
history_dnn = model_dnn.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test,y_test))

In [ ]:
epochs = 100
batch_size = 100
n_splits = 5
cv_scores = []

X_reset = X.reset_index(drop=True)
y_reset = Y.reset_index(drop=True)

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_reset)):
    print(f"\n===== Fold {fold+1} =====")

    # Séparation des données
    X_tr, X_val = X_reset.iloc[train_idx], X_reset.iloc[val_idx]
    y_tr, y_val = y_reset.iloc[train_idx], y_reset.iloc[val_idx]

    # Recréer le modèle à chaque fold
    model = dnn_model(learning,dropout,X.shape[1])

    # Entraînement
    history_CV = model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Évaluation sur le fold
    scores_CV = model.evaluate(X_val, y_val, verbose=0)
    acc = scores_CV[1]
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")

    cv_scores.append(acc)

# Conserver l’historique du dernier fold (utile pour tracer les courbes)
history = history_CV

# Résultats finaux
print("\n===== Résultats finaux =====")
print("CV Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores)*100, np.std(cv_scores)*100))


In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy
ax1.plot(history_dnn.history['accuracy'], label='Train Accuracy')
ax1.plot(history_dnn.history['val_accuracy'], label='Val Accuracy')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# Loss
ax2.plot(history_dnn.history['loss'], label='Train Loss')
ax2.plot(history_dnn.history['val_loss'], label='Val Loss')
ax2.set_title('Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()

plt.tight_layout()
plt.show()

#Evaluation modèle DNN
